#### Vamos a entrenar una red siamesa pero usando ahora recortes particulares de las caras (Vamos a probar si esto es capaz de mejorar el rendimiento del modelo) 

### Cargamos las bibliotecas necesarias

In [1]:
import os
import random
import numpy as np
from itertools import combinations
import tensorflow as tf
import cv2
from matplotlib import pyplot as plt
from tensorflow.keras.metrics import Precision, Recall, Accuracy


# Importamos algunas dependencias fundamentales de la API de tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

import mtcnn
from mtcnn.mtcnn import MTCNN


2024-10-22 15:02:33.613905: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


#### Creamos los directorios para almacenar los recortes de las caras. Cada recorte de cara se redimensiona a tamaño 224x224

In [2]:
#Obtenemos la ruta actual
ruta_actual = os.getcwd()

#Definimos las rutas de los directorios que vamos a crear
ruta_recortes_caras = os.path.join(ruta_actual,"recortes_224")
ruta_recortes_entrenamiento = os.path.join(ruta_recortes_caras,"entrenamiento")
ruta_recortes_test = os.path.join(ruta_recortes_caras,"test")

#Creamos los directorios para almacenar las recortes de las caras de las imágenes y los redimensionamos a tamaño 105x105
try:
    os.mkdir(ruta_recortes_caras)
except:
    print("La carpeta ya existe")

#Creamos la carpeta para almacenar los recortes de entrenamiento
try:
    os.mkdir(ruta_recortes_entrenamiento)
except:
    print("La carpeta ya existe")

#Creamos la carpeta para almacenar los recortes de test
try:
    os.mkdir(ruta_recortes_test)
except:
    print("La carpeta ya existe")



#### Definimos los directorios originales que contienen las imágenes

In [3]:
ruta_original_entrenamiento = os.path.join(ruta_actual,"vggface2_train","train")
ruta_original_test = os.path.join(ruta_actual,"vggface2_test","test")

lista_personas_entrenamiento = os.listdir(ruta_original_entrenamiento) 
lista_personas_test = os.listdir(ruta_original_test)

#Se hara un resize de las imágenes al tamaño de entrada de la red.
tamano_resize = 224
contador = 0 #Contamos las imágenes de las que no se pudo extraer cara

#Extraemos los recortes para el conjunto de entrenamiento
for persona in lista_personas_entrenamiento:

    #Creamos la carpeta para almacenar los recortes de las caras
    ruta_recortes_entrenamiento_persona = os.path.join(ruta_recortes_entrenamiento,persona) 
    try:
        os.mkdir(ruta_recortes_entrenamiento_persona)
    except:
        print("La carpeta ya estaba creada")
    
    lista_imagenes_personas = os.listdir(os.path.join(ruta_original_entrenamiento,persona))

    for nombre_imagen in lista_imagenes_personas:
        #ruta a la imagen
        ruta_imagen = os.path.join(ruta_original_entrenamiento,persona,nombre_imagen)
        
        # Cargamos la imagen usando OpenCV
        imagen = cv2.imread(ruta_imagen)

        #Convertimos la imagen de BGR a RGB
        imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

        # Inicializamos el detector de caras de MTCNN
        detector = MTCNN()

        # Detectamos las caras en la imagen
        resultados = detector.detect_faces(imagen_rgb)
        
        # Verificar si se detectó al menos una cara
        if len(resultados) > 0:
        # Obtenemos las coordenadas de la primera cara detectada (rectángulo)
            x, y, ancho, alto = resultados[0]['box']
    
        # Extraemos la región de la cara de la imagen original
            cara = imagen_rgb[y:y+alto, x:x+ancho]
            cara_redimensionada = cv2.resize(cara, (tamano_resize, tamano_resize))

            # Convertimos la imagen de nuevo a BGR para guardar con OpenCV
            cara_redimensionada_bgr = cv2.cvtColor(cara_redimensionada, cv2.COLOR_RGB2BGR)

            ruta_guardado = os.path.join(ruta_recortes_entrenamiento_persona,nombre_imagen)
            cv2.imwrite(ruta_guardado, cara_redimensionada_bgr)

        else:
            contador=contador+1

print(f"Hubo {contador} imágenes de entrenamiento en las que no se pudiero extraer recortes de las caras")

2024-10-22 15:02:34.841078: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-22 15:02:34.858707: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-22 15:02:34.858812: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-22 15:02:34.859151: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

Hubo 15140 imágenes de entrenamiento en las que no se pudiero extraer recortes de las caras


In [4]:
#Extraemos los recortes para el conjunto de test

contador = 0

for persona in lista_personas_test:

    #Creamos la carpeta para almacenar los recortes de las caras
    ruta_recortes_test_persona = os.path.join(ruta_recortes_test,persona) 
    try:
        os.mkdir(ruta_recortes_test_persona)
    except:
        print("La carpeta ya estaba creada")
    
    lista_imagenes_personas = os.listdir(os.path.join(ruta_original_test,persona))

    for nombre_imagen in lista_imagenes_personas:
        #ruta a la imagen
        ruta_imagen = os.path.join(ruta_original_test,persona,nombre_imagen)
        
        # Cargamos la imagen usando OpenCV
        imagen = cv2.imread(ruta_imagen)

        #Convertimos la imagen de BGR a RGB
        imagen_rgb = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

        # Inicializamos el detector de caras de MTCNN
        detector = MTCNN()

        # Detectamos las caras en la imagen
        resultados = detector.detect_faces(imagen_rgb)
        
        # Verificar si se detectó al menos una cara
        if len(resultados) > 0:
        # Obtenemos las coordenadas de la primera cara detectada (rectángulo)
            x, y, ancho, alto = resultados[0]['box']
    
        # Extraemos la región de la cara de la imagen original
            cara = imagen_rgb[y:y+alto, x:x+ancho]
            cara_redimensionada = cv2.resize(cara, (tamano_resize, tamano_resize))

            # Convertimos la imagen de nuevo a BGR para guardar con OpenCV
            cara_redimensionada_bgr = cv2.cvtColor(cara_redimensionada, cv2.COLOR_RGB2BGR)

            ruta_guardado = os.path.join(ruta_recortes_test_persona,nombre_imagen)
            cv2.imwrite(ruta_guardado, cara_redimensionada_bgr)

        else:
            contador=contador+1

print(f"Hubo {contador} imágenes de test en las que no se pudieron extraer los recortes de las caras")


Hubo 983 imágenes de test en las que no se pudieron extraer los recortes de las caras


#### Contamos el mínimo número de imágenes disponibles para cualquiera de las clases.

In [5]:
minimo = 100000 #Ninguna clase tiene tantas imágenes
for elemento in os.listdir(ruta_recortes_entrenamiento):
    numero_imagenes_elemento = len(os.listdir(os.path.join(ruta_recortes_entrenamiento,elemento)))

    if minimo > numero_imagenes_elemento:
        minimo = numero_imagenes_elemento

print(f"El mínimo de imágenes por clase es {minimo}")

El mínimo de imágenes por clase es 87


In [6]:
minimo = 100000 #Ninguna clase tiene tantas imágenes
for elemento in os.listdir(ruta_recortes_test):
    numero_imagenes_elemento = len(os.listdir(os.path.join(ruta_recortes_test,elemento)))

    if minimo > numero_imagenes_elemento:
        minimo = numero_imagenes_elemento

print(f"El mínimo de imágenes por clase es {minimo}")

El mínimo de imágenes por clase es 98


#### Para cada persona del dataset tenemos al menos 87 imágenes de su cara.